# PURE-TF (1.13.0rc2) FaceAttribute

In [1]:
import tensorflow as tf
import numpy as np
import random
import os

# tf.enable_eager_execution()
AUTOTUNE = tf.data.experimental.AUTOTUNE

## 1. Dataset

In [2]:
NUM_IMGS = 202599

def load_labels(labels_path):
    f = open(labels_path)
    # line 1: number of images
    num_imgs = int(f.readline())
    # line 2: attribute names, 40 in total
    attr_names = f.readline().split()
    # line 3 to end: 00xx.jpg -1 1 -1 1 ...
    labels = []
    for i in range(num_imgs):
        labels.append(list(map(np.float32, f.readline().split()[1:])))
    labels = np.array(labels)
    labels[labels<0] = 0
    return labels

def load_imgs(imgs_dir):
    img_paths = os.listdir(imgs_dir)
    img_paths.sort()
    for i in range(len(img_paths)):
        img_paths[i] = os.path.join(imgs_dir,img_paths[i])
    return img_paths

def preprocess(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    # uint8 range: [0,255]
    img = tf.image.resize(img, [192, 160])
    # new range: [-128,127]
    img -= 128
    img = tf.image.convert_image_dtype(img, tf.int8, saturate=True)
    return img    

def parse(x):
    result = tf.parse_tensor(x, out_type=tf.int8)
    result = tf.reshape(result, (192,160,3))
    return result

if os.path.exists('../dataset/tfrec') == False:
    os.mkdir('../dataset/tfrec')
    imgs_dir  = '../dataset/img_align_celeba'
    img_paths = load_imgs(imgs_dir)
    ds_imgs = tf.data.Dataset.from_tensor_slices(img_paths).map(preprocess)
    ds_imgs_serialized = ds_imgs.map(tf.serialize_tensor)
    tfrec = tf.data.experimental.TFRecordWriter('../dataset/tfrec/imgs.tfrec')
    tfrec.write(ds_imgs_serialized)
else: 
    ds_imgs = tf.data.TFRecordDataset('../dataset/tfrec/imgs.tfrec')
    ds_imgs = ds_imgs.map(parse, num_parallel_calls=AUTOTUNE)

labels_path = '../dataset/list_attr_celeba.txt'
ds_labels = tf.data.Dataset.from_tensor_slices(load_labels(labels_path))

ds_celeba = tf.data.Dataset.zip((ds_imgs, ds_labels))
ds_celeba = ds_celeba.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=8192*4))
ds_celeba = ds_celeba.batch(32).prefetch(AUTOTUNE)
print(ds_celeba)


<DatasetV1Adapter shapes: ((?, 192, 160, 3), (?, 40)), types: (tf.int8, tf.float32)>


## 2. Train with Keras 

In [ ]:
mnet = tf.keras.applications.mobilenet.MobileNet(input_shape=(192,160,3),alpha=0.5,include_top=False,weights=None,pooling='avg')
mnet = tf.keras.Sequential([mnet,tf.keras.layers.Dense(40,activation='sigmoid',name='top_dense')], name='mnet_050_faceattr')
mnet.summary()

mnet.compile(optimizer=tf.keras.optimizers.RMSprop(),
            loss='binary_crossentropy',
            metrics=['binary_accuracy'])

mnet.fit(ds_celeba, epochs=5, steps_per_epoch=20, verbose=2)

## 3. Train with pure TF

### 3.1 Build a mobilenet

In [2]:
def mobilenet_v1(tensor_in, num_classes, depth_multiplier, is_training):
    """
    Constructs a Mobilenet V1 base convnet
    
    Args:
        tensor_in: a tensor of shape [NHWC]
        num_classes: number of channels of the final dense layer
        depth_multiplier: multiplier for number of channels, 
            should be 0.25, 0.5, 0.75 or 1.0
        is_training: the model is constructed for training or not
        
    Returns:
        logits: output tensor
    """
    
    # list of dicts specifying the base net architecture
    MOBILENET_V1_BASE_DEFS = [
        {'layer':'conv2d', 'name':'Conv_0',  'stride':2, 'depth':32  },
        {'layer':'convds', 'name':'Conv_1',  'stride':1, 'depth':64  },
        {'layer':'convds', 'name':'Conv_2',  'stride':2, 'depth':128 },
        {'layer':'convds', 'name':'Conv_3',  'stride':1, 'depth':128 },
        {'layer':'convds', 'name':'Conv_4',  'stride':2, 'depth':256 },
        {'layer':'convds', 'name':'Conv_5',  'stride':1, 'depth':256 },
        {'layer':'convds', 'name':'Conv_6',  'stride':2, 'depth':512 },
        {'layer':'convds', 'name':'Conv_7',  'stride':1, 'depth':512 },
        {'layer':'convds', 'name':'Conv_8',  'stride':1, 'depth':512 },
        {'layer':'convds', 'name':'Conv_9',  'stride':1, 'depth':512 },
        {'layer':'convds', 'name':'Conv_10', 'stride':1, 'depth':512 },
        {'layer':'convds', 'name':'Conv_11', 'stride':1, 'depth':512 },
        {'layer':'convds', 'name':'Conv_12', 'stride':2, 'depth':1024},
        {'layer':'convds', 'name': 'Conv_13', 'stride':1, 'depth':1024}
    ]
    
    # hyperparams to use
    activation_fn = tf.nn.relu6
    normalizer_fn=tf.contrib.slim.batch_norm
    normalizer_params = {
        'is_training': is_training,
        'center': True, 
        'scale': True, 
        'decay': 0.9997, 
        'epsilon': 0.001, 
        'updates_collections': tf.GraphKeys.UPDATE_OPS
    }
    weights_initializer = tf.truncated_normal_initializer(stddev=0.09)
    weights_regularizer = tf.contrib.layers.l2_regularizer(0.00004)
    
    with tf.variable_scope('MobilenetV1', [tensor_in]):
        net = tensor_in
        # conv layers
        for layer_def in MOBILENET_V1_BASE_DEFS:
            if layer_def['layer']=='conv2d':
                net = tf.contrib.slim.conv2d( net,
                                              num_outputs=layer_def['depth']*depth_multiplier,
                                              kernel_size=[3,3],
                                              stride=layer_def['stride'],
                                              activation_fn=activation_fn,
                                              normalizer_fn=normalizer_fn,
                                              normalizer_params=normalizer_params,
                                              weights_initializer=weights_initializer,
                                              weights_regularizer=weights_regularizer,
                                              scope=layer_def['name'])
            elif layer_def['layer'] == 'convds':
                # depthwise conv
                net = tf.contrib.slim.separable_conv2d(net, 
                                                       num_outputs=None, # to skip pointwise stage
                                                       kernel_size=[3,3], 
                                                       stride=layer_def['stride'], 
                                                       activation_fn=activation_fn,
                                                       normalizer_fn=normalizer_fn,
                                                       normalizer_params=normalizer_params,
                                                       weights_initializer=weights_initializer,
                                                       scope=layer_def['name']+'_depthwise')
                # pointwise conv
                net = tf.contrib.slim.conv2d(net,
                                             num_outputs=layer_def['depth']*depth_multiplier,
                                             kernel_size=[1,1],
                                             activation_fn=activation_fn,
                                             normalizer_fn=normalizer_fn,
                                             normalizer_params=normalizer_params,
                                             weights_initializer=weights_initializer,
                                             weights_regularizer=weights_regularizer,
                                             scope=layer_def['name']+'_pointwise')
                
            else:
                raise ValueError('Unsupported layer type'+layer_def['layer'])
            
        # top layers
        convout_shape = net.get_shape().as_list()
        net = tf.contrib.slim.avg_pool2d(net, [convout_shape[1],convout_shape[2]], padding='VALID', scope='AvgPool')
        net = tf.contrib.slim.dropout(net, keep_prob=0.999, is_training=is_training, scope='Dropout')
        logits = tf.contrib.slim.conv2d(net, 
                                num_outputs=num_classes, 
                                kernel_size=[1,1], 
                                activation_fn=None,
                                normalizer_fn=None, 
                                scope='Dense')
        logits = tf.squeeze(logits, axis=[1,2], name='Squeeze')
        return logits

### 3.2 Create Train-Eval Loop

In [3]:
def train_eval_loop():
    # create a session
    sess = tf.InteractiveSession()
    g = tf.get_default_graph()

    # create placeholders for images and labels
    images_placeholder = tf.placeholder(tf.float32, shape=[None, 192, 160, 3], name='images')
    labels_placeholder = tf.placeholder(tf.float32, shape=[None, 40], name='labels')

    # create model
    logits = mobilenet_v1(images_placeholder, num_classes=40, depth_multiplier=0.5, is_training=True)

    # define loss
    loss = tf.losses.sigmoid_cross_entropy(labels_placeholder, logits)

    # create quantized training graph
    tf.contrib.quantize.create_training_graph(quant_delay=0)

    # learning rate
    global_step = tf.Variable(tf.constant(0))
    learning_rate = tf.train.exponential_decay(0.01, global_step, 450, 0.8, staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    
    
    
train_eval_loop()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Skipping quant after MobilenetV1/Conv_0/add_fold
INFO:tensorflow:Skipping quant after MobilenetV1/Conv_1_depthwise/add_fold
INFO:tensorflow:Skipping quant after MobilenetV1/Conv_1_pointwise/add_fold
INFO:tensorflow:Skipping quant after MobilenetV1/Conv_2_depthwise/add_fold
INFO:tensorflow:Skipping quant after MobilenetV1/Conv_2_pointwise/add_fold
INFO:tensorflow:Skipping quant after MobilenetV1/Conv_3_depthwise/add_fold
INFO:tensorflow:Skipping quant after MobilenetV1/Conv_3_pointwise/add_fold
INFO:tensorflo